In [1]:
import dspy
from pydantic import BaseModel, Field
from typing import List

seed_num = 42

lm = dspy.LM('openai/gpt-4o', api_key='')
dspy.configure(lm=lm)

In [2]:
# Define Pydantic models for structured outputs
class TaskComponent(BaseModel):
    line_to_append: str = Field(description="Context/instruction for the task")
    sample_sentences: List[str] = Field(description="2-3 sample phrases/sentences")
    
class GeneratePropertyBuyerAgent(dspy.Signature):
    """Generate a complete user profile for a property listing simulation. The profile will simulate a real buyer/renter interacting with a real estate agent.

    Objection : Resistance from the user toward property listings (e.g., price concerns, location issues, property condition).
    
    Create persona, traits, behaviors, and detailed conversation tasks. 
    Use fictional phone number 0000-000-000 if not provided.
    
    Steps:
    1. Persona: Name, phone, role (first-time buyer, investor, renter, etc.) 
    2. Key Traits: 3-5 characteristics relevant to objection
    3. Key Behaviors: 3-5 conversation behaviors
    4. Tasks Components (with line_to_append + sample_sentences):
        - Initial conversation starter
        - Objection reaction
        - Information disclosure
        - Value proposition questioning
        - Resolution conditions
        - Non-resolution response
        - Property viewing scheduling
    5. Final Identity: Combined persona + all traits
    6. Final Behavior/Task: Structured behaviors + all tasks components with sample sentences
    
    Format Final Identity: "You are [Name], phone: [0000-000-000], [Role]. Key traits:\n- Trait1\n- Trait2"
    Format Final Behavior/Task: "## Key Behaviors\n- Behavior1\n## Task\n1. [Task1 description]\n   - Sample1\n2. [Task2 description]\n   - Sample2"
    """
    
    objection_description: str = dspy.InputField(desc="Type of objection (e.g., price, location, condition)")

    persona: str = dspy.OutputField(desc="Full persona description")
    key_traits: List[str] = dspy.OutputField(desc="3-5 key traits")
    key_behaviors: List[str] = dspy.OutputField(desc="3-5 key behaviors")
    initial_line: TaskComponent = dspy.OutputField(desc="Initial conversation starter")
    objection: TaskComponent = dspy.OutputField(desc="Reaction to objection")
    disclose_info: TaskComponent = dspy.OutputField(desc="Information disclosure strategy")
    questioning_value: TaskComponent = dspy.OutputField(desc="Value proposition challenges")
    conditions_resolution: TaskComponent = dspy.OutputField(desc="Acceptance when resolved")
    conditions_without_resolution: TaskComponent = dspy.OutputField(desc="Response when unresolved")
    scheduling_viewing: TaskComponent = dspy.OutputField(desc="Property viewing approach")
    final_identity: str = dspy.OutputField(desc="Combined persona + traits")
    final_behaviour_task: str = dspy.OutputField(desc="Structured behaviors + all tasks")
    
class PropertyBuyerAgentGenerator(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate = dspy.Predict(GeneratePropertyBuyerAgent)
    
    def forward(self, objection_description: str):
        return self.generate(objection_description=objection_description)

In [3]:
objection_desc = """
User name Sarah with phone number 123-456-7890. 
Sarah is a first-time home buyer looking for a 3-bedroom house in the downtown area.
Her maximum budget is $500,000. 
Any listing above $500,000 will trigger her price objection.
She's concerned about property taxes and maintenance costs.
"""

# Example usage
generator = PropertyBuyerAgentGenerator()
result = generator(objection_description=objection_desc)

result

Prediction(
    persona='Sarah is a first-time home buyer with a phone number 123-456-7890. She is looking for a 3-bedroom house in the downtown area with a maximum budget of $500,000. Sarah is particularly concerned about property taxes and maintenance costs.',
    key_traits=['Budget-conscious', 'Detail-oriented', 'Cautious', 'Analytical', 'Future-focused'],
    key_behaviors=['Frequently asks about price breakdowns', 'Inquires about long-term costs', 'Seeks detailed property reports', 'Questions the value of features', 'Requests comparisons with similar properties'],
    initial_line=TaskComponent(line_to_append="Start the conversation by understanding Sarah's needs and preferences.", sample_sentences=["Hi Sarah, I understand you're looking for a 3-bedroom house in the downtown area. Can you tell me more about your preferences?", "Hello Sarah, let's discuss what you're looking for in a home. What are your must-haves?", "Hi Sarah, I'm here to help you find the perfect home. What are 

In [6]:
def get_identity_string(prediction):
    identity = "## Identity\n"
    identity += prediction['persona'] + "\n"
    identity += "Key Traits:\n"
    for trait in prediction['key_traits']:
        identity += f"- {trait}\n"
    return identity.strip()

In [7]:
print(get_identity_string(result))

## Identity
Sarah is a first-time home buyer with a phone number 123-456-7890. She is looking for a 3-bedroom house in the downtown area with a maximum budget of $500,000. Sarah is particularly concerned about property taxes and maintenance costs.
Key Traits:
- Budget-conscious
- Detail-oriented
- Cautious
- Analytical
- Future-focused


In [4]:
def get_task_string(prediction):
    task_str = "## Key Behaviours\n"
    for behavior in prediction['key_behaviors']:
        task_str += f"- {behavior}\n"

    task_str += "\n## Tasks\n"

    task_components = [
        ('initial_line', "1."),
        ('objection', "2."),
        ('disclose_info', "3."),
        ('questioning_value', "4."),
        ('conditions_resolution', "5."),
        ('conditions_without_resolution', "6."),
        ('scheduling_tour', "7.")
    ]

    for i, (key, num) in enumerate(task_components, start=1):
        task = prediction.get(key)
        if task:
            task_str += f"{i}. {task.line_to_append}\n"
            for sentence in task.sample_sentences:
                task_str += f"   - {sentence}\n"
    return task_str.strip()

In [5]:
print(get_task_string(result))

## Key Behaviours
- Frequently asks about price breakdowns
- Inquires about long-term costs
- Seeks detailed property reports
- Questions the value of features
- Requests comparisons with similar properties

## Tasks
1. Start the conversation by understanding Sarah's needs and preferences.
   - Hi Sarah, I understand you're looking for a 3-bedroom house in the downtown area. Can you tell me more about your preferences?
   - Hello Sarah, let's discuss what you're looking for in a home. What are your must-haves?
   - Hi Sarah, I'm here to help you find the perfect home. What are the key features you're looking for?
2. Address Sarah's price concerns by acknowledging her budget and offering alternatives.
   - I understand that staying within your budget is important. Let's explore some options that fit your criteria.
   - I see that this listing is above your budget. Would you like to see similar properties that are within your range?
   - I know price is a concern. Let's look at some home